In [55]:
import pandas as pd

In [56]:
df = pd.read_csv('dataset_unlabeled_penyisihan_bdc_2024.csv', delimiter=';')

In [57]:
sub = pd.read_csv('submission_1.csv', delimiter=';')

In [58]:
import re
def process_text(text):
  # Counter
  def count_at(text):
    return text.count('@')
  def count_hashtag(text):
    return text.count('#')
  #Preprocess
  def retrieve_hashtags(text):
    return " ".join(re.findall(r'#\S+', text))
  def remove_retweet(text):
    text = re.sub(r"@\S+ ", "", text)
    return re.sub(r"@\S+$", "", text)
  def remove_hashtag(text):
    text = re.sub(r"#\S+ ", "", text)
    return re.sub(r"#\S+$", "", text)
  def remove_unwanted_string(text):
    # Remove substrings containing unwanted characters
    text = re.sub(r'\S*[ÃÂš°µ]\S*', '', text)
    return text.strip()
  def remove_rebrackets(text):
    return re.sub(r'\[RE[^\]]*\]', '', text)
  def remove_urls(text):
      # Remove URLs starting with https
      return re.sub(r'https?://\S+', '', text)
  def preprocess_hashtag(text):
    if(len(text)==0):
      return 0,0,0
    texted = text.split(" ")
    #print(texted)
    hashtag_1 = ['01','amin','anies','imin','muhaimin','kosongsatu','1']
    hashtag_2 = ['02','prabowo','gibran','kosongdua','indonesiasentris','2']
    hashtag_3 = ['03','ganjar','mahfud','kosongtiga','jnk','3']
    count_tag_1 = 0
    count_tag_2 = 0
    count_tag_3 = 0
    for text in texted :
      for word_to_match in hashtag_1:
        match = re.search(word_to_match, text)
        # Jika kecocokan ditemukan
        if match:
          count_tag_1 = count_tag_1 + 1
          # print(f"'{word_to_match}' matches with '{text}'")
      for word_to_match in hashtag_2:
        match = re.search(word_to_match, text)
        # Jika kecocokan ditemukan
        if match:
          count_tag_2 = count_tag_2 + 1
          # print(f"'{word_to_match}' matches with '{text}'")
      for word_to_match in hashtag_3:
        match = re.search(word_to_match, text)
        # Jika kecocokan ditemukan
        if match:
          count_tag_3 = count_tag_3 + 1
          # print(f"'{word_to_match}' matches with '{text}'")
    return count_tag_1, count_tag_2,count_tag_3
  def count_paslon(text):
    #Process the hashtag first
    hashtag_text = retrieve_hashtags(text).lower()
    # print(hashtag_text)
    count_tag_1,count_tag_2,count_tag_3 = preprocess_hashtag(hashtag_text)
    text = remove_hashtag(text)
    # print(text)
    text_split = re.sub(r'[^\w\s]', '', text.lower()).split()
    # print(text_split)
    paslon_1 = ['anies','muhaimin','imin','anis','01','waktunya','perubahan','amin' , 'asalbukanprabowo']
    paslon_2 = ['prabowo','gibran','rakabuming','02','menhan','lanjutkan','gerindra','masbowo','anakmuda']
    paslon_3 = ['ganjar','pranowo','mahfud','03','pdi','asalbukanprabowo','ganjarmahfud2024','ganjarmahfudsatset']
    counter_01 = 0
    counter_02 = 0
    counter_03 = 0
    for x in text_split:
      for word_to_match in paslon_1:
        match = re.search(word_to_match, x)
        if match:
          counter_01 = counter_01 + 1
      for word_to_match in paslon_2:
        match = re.search(word_to_match, x)
        if match:
          counter_02 = counter_02 + 1
      for word_to_match in paslon_3:
        match = re.search(word_to_match, x)
        if match:
          counter_03 = counter_03 + 1
    return " ".join(text_split),counter_01,counter_02,counter_03,count_tag_1,count_tag_2,count_tag_3,hashtag_text


  count_att = count_at(text)
  text = remove_retweet(text)
  text = remove_urls(text)
  text = remove_unwanted_string(text)
  text = remove_rebrackets(text)
  count_hashtag = count_hashtag(text)
  text, count_1,count_2,count_3,count_tag_1,count_tag_2,count_tag_3,hashtag_text = count_paslon(text)
  text = remove_hashtag(text)
  return text,count_att,count_hashtag,count_1,count_2,count_3,count_tag_1,count_tag_2,count_tag_3,hashtag_text

In [59]:
df['Text'] = df['Text'].str.replace('RT', '')
df[['text', 'retweet', 'hashtag', 'count_1', 'count_2', 'count_3','count_tag_1','count_tag_2','count_tag_3','hashtag_text']] = df['Text'].apply(lambda x: pd.Series(process_text(x)))

In [60]:
sub['Text'] = df['text']

In [61]:
sub

,IDText,Kelas,Text
0,TXT0001,Politik,lu mau org2 prodemokrasi di negara ini bisa pu...
1,TXT0002,Politik,prabowo ditanya soal hutang luar negeri dia me...
2,TXT0003,Ideologi,kiki_daliyo ganjar pranowo itulah beliau sosok...
3,TXT0004,Politik,prabowo gibran yang bisa melakukan itu semua d...
4,TXT0005,Ekonomi,lah justru yg gak nyambung junjungan elu aomkm...
...,...,...,...
995,TXT0996,Ekonomi,bikin bangga deh ganjarmahfud mau alokasikan s...
996,TXT0997,Politik,pak jokowi sebelum pilpres 2024 berbesar hati ...
997,TXT0998,Politik,sbaiknya si gemot nga usah ikutan debat dehnga...
998,TXT0999,Politik,kebiasaan merembuk atau bermusyawarah jadi gay...


In [62]:
words = ['gen z', 'berpendidikan', 'muda', 'pemilih muda', 'etnis', 'generasi tua', 'generasi muda', 'kalangan atas', 'kalangan menengah']
# sub.loc[sub['Text'].str.contains('|'.join(words)), 'Kelas'] = 'Demografi'

In [63]:
sub['Kelas'].value_counts()

Kelas
Politik                    780
Ekonomi                    105
Pertahanan dan Keamanan     52
Sosial Budaya               30
Ideologi                    21
Sumber Daya Alam            10
Demografi                    1
Geografi                     1
Name: count, dtype: int64

In [75]:
sub.loc[(sub['Text'].str.contains('|'.join(words))) & (sub['Kelas'] == 'Politik'), 'Kelas'] = 'Demografi'

In [71]:
sub.loc[sub['Text'].str.contains('pulau'), 'Kelas'] = 'Geografi'

In [76]:
for elm in sub[sub['Kelas'] == 'Demografi']['Text']:
    print(elm + '\n' + '='*100)

kampanye politik anies dalam pilgub dki mempengaruhi dinamika masyarakat di mana generasi tua memendam kebencian yang sulit diperbaiki oleh generasi muda berdampak pada kohesi sosial antar kelompok usia dalam populasi
abah online meroket figur pak anies semakin populer di kalangan anak anak milenial dan anakanak genz gagasan dan gaya komunikasi pak anies tentu berhasil memikat anak anak muda terutama anak anak genz dan ini memberi warna baru dalam kancah berpoltikan
sampai kapan kubu prabowo bodohi rakyat oleh tiand raharjo sangat menggelitik sebenarnya melihat kubu prabowogibran dengan masif mengampanyekan pilpres satu putaran dengan alasan menghemat uang negara ini tentu pembodohan sekaligus pengkerdilan yang luar biasa terhadap hakhak rakyat kemungkinan prabowo kalah di putaran dua memang sangat besar dan narasi itu mudah kita tebak hanya untuk menutupi kepanikan yang makin mendera kubu mereka bayangkan saja segala infrastruktur negara sudah dipakai dari mulai aparat hingga penjabat

In [77]:
sub['Kelas'].value_counts()

Kelas
Politik                    761
Ekonomi                    104
Pertahanan dan Keamanan     51
Sosial Budaya               30
Ideologi                    21
Demografi                   20
Sumber Daya Alam             9
Geografi                     4
Name: count, dtype: int64

In [67]:
asli = pd.read_csv('df.csv', delimiter=';')

In [68]:
asli['label'].value_counts()

label
Politik                    2972
Sosial Budaya               587
Pertahanan dan Keamanan     400
Ideologi                    400
Ekonomi                     367
Sumber Daya Alam            192
Demografi                    62
Geografi                     20
Name: count, dtype: int64

In [78]:
cekkk = pd.read_csv('submission_1.csv', delimiter=';')

In [79]:
cekkk['Kelas'].value_counts()

Kelas
Politik                    780
Ekonomi                    105
Pertahanan dan Keamanan     52
Sosial Budaya               30
Ideologi                    21
Sumber Daya Alam            10
Demografi                    1
Geografi                     1
Name: count, dtype: int64

In [84]:
sub[['IDText', 'Kelas']].to_csv('SD2024040000318.csv', index=False)

In [86]:
sub['Text'][0]

'lu mau org2 prodemokrasi di negara ini bisa punya kesempatan lagi buat ngelawan oligarki di indo pilih amin tpi jangan pilih pks pilih partai buruh jangan pilih prabowo dan semua partai pengusungnya pilih gama tpi jangan pilih pdip pilih partai buruh'